# Compare GP Kernels

To create a run, use `pixi run python celeri/scripts/celeri_solve.py /Users/eliot/wna/data/config/wna_config_00001.json --mcmc-seed=42`

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [10]:
import matplotlib.collections as mc
import matplotlib.pyplot as plt
import numpy as np

import celeri
from celeri.plot import plot_mesh

In [11]:
orig_run_dir = "/Users/eliot/wna/runs/0000000045"
new_run_dir = "/Users/eliot/wna/runs/0000000065"

new_estimation = celeri.Estimation.from_disk(new_run_dir)
new_trace = new_estimation.mcmc_trace

orig_estimation = celeri.Estimation.from_disk(orig_run_dir)
orig_trace = orig_estimation.mcmc_trace

### Original Run

In [ ]:
kind = "ss"
mesh_idx = 0

vmin = -5
vmax = 5

fig, axes = plt.subplots(2, 5, figsize=(12, 6))

for ax, draw in zip(axes.flat, range(0, 1000, 100)):
    pc = plot_mesh(
        orig_estimation.model.meshes[mesh_idx],
        fill_value=orig_trace.posterior[f"coupling_{mesh_idx}_{kind}"].isel(
            chain=0, draw=draw
        ),
        ax=ax,
        set_limits=True,
    )

cbar = fig.colorbar(
    pc,
    ax=axes,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle(f"10 posterior draws of {kind} coupling on mesh {mesh_idx}");

### New Run

In [ ]:
kind = "ss"
mesh_idx = 0

vmin = -5
vmax = 5

fig, axes = plt.subplots(2, 5, figsize=(12, 6))

for ax, draw in zip(axes.flat, range(0, 1000, 100)):
    pc = plot_mesh(
        new_estimation.model.meshes[mesh_idx],
        fill_value=new_trace.posterior[f"coupling_{mesh_idx}_{kind}"].isel(
            chain=0, draw=draw
        ),
        ax=ax,
        set_limits=True,
    )

cbar = fig.colorbar(
    pc,
    ax=axes,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle(f"10 posterior draws of {kind} coupling on mesh {mesh_idx}");

### Plot the Eigenmodes

In [ ]:
kind = "strike_slip"
mesh_idx = 0
vmin = -0.1
vmax = 0.1

mesh = new_estimation.model.meshes[mesh_idx]
fig, axes = plt.subplots(2, 5, figsize=(12, 8))

from celeri.plot import plot_mesh

for ax, mode in zip(axes.flat, range(0, 10)):
    pc = plot_mesh(mesh, fill_value=mesh.eigenvectors[:, mode], ax=ax)

cbar = fig.colorbar(
    pc,
    ax=axes,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)

## Draws from prior

In [ ]:
center = 0
cmap = "seismic"
kind = "dip_slip"
mesh_idx = 0
mesh = new_estimation.model.meshes[mesh_idx]

np.random.seed(42)
coefs = np.random.normal(size=(mesh.eigenvectors.shape[1], 10))
coefs = coefs * 10
fig, axes = plt.subplots(2, 5, figsize=(12, 8))

for ax, mode in zip(axes.flat, range(0, 10)):
    field = mesh.eigenvectors @ coefs[:, mode]

    pc = plot_mesh(
        new_estimation.model.meshes[mesh_idx],
        fill_value=field,
        ax=ax,
        cmap=cmap,
        set_limits=True,
    )

cbar = fig.colorbar(
    pc,
    ax=ax,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle("Random Draws from Original Prior (Strike Slip)", fontsize=16);

In [ ]:
cmap = "seismic"
kind = "dip_slip"
mesh_idx = 0
mesh = new_estimation.model.meshes[mesh_idx]

np.random.seed(42)
coefs = np.random.normal(size=(mesh.eigenvectors.shape[1], 10))
coefs = coefs * 10 * np.sqrt(mesh.eigenvalues)[:, None]

fig, axes = plt.subplots(2, 5, figsize=(12, 8))

for ax, draw in zip(axes.flat, range(0, 10)):
    field = mesh.eigenvectors @ coefs[:, draw]

    pc = plot_mesh(
        new_estimation.model.meshes[mesh_idx],
        fill_value=field,
        ax=ax,
        cmap=cmap,
        set_limits=True,
    )

cbar = fig.colorbar(
    pc,
    ax=ax,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle("Random Draws from Matern 1/2 Prior", fontsize=16);

### Increasing the number of eigenmodes used with original approach 

In [17]:
from celeri.mesh import _get_eigenvalues_and_eigenvectors

eigenvalues, eigenvectors = _get_eigenvalues_and_eigenvectors(
    n_eigenvalues=1500,
    x=new_estimation.model.meshes[0].x_centroid,
    y=new_estimation.model.meshes[0].y_centroid,
    z=new_estimation.model.meshes[0].z_centroid,
)

Original

In [ ]:
mesh = new_estimation.model.meshes[0]
np.random.seed(42)
fig, axes = plt.subplots(2, 5, figsize=(12, 8))
n_eigs = [1, 5, 10, 25, 50, 100, 250, 500, 1000, 1500]
coefs = np.random.normal(size=1500)
for ax, mode in zip(axes.flat, n_eigs):
    field = eigenvectors[:, :mode] @ coefs[:mode]

    pc = plot_mesh(mesh, fill_value=field, ax=ax, set_limits=True)
    # Add a caption with the value of n_eigs for each subplot
    ax.set_title(f"n_eigs = {mode}", fontsize=10)

cbar = fig.colorbar(
    pc,
    ax=ax,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle("Uniformly-Weighted Eigenmode Prior", fontsize=16);

New

In [ ]:
mesh = new_estimation.model.meshes[0]
np.random.seed(42)
fig, axes = plt.subplots(2, 5, figsize=(12, 8))
n_eigs = [1, 5, 10, 25, 50, 100, 250, 500, 1000, 1500]
coefs = np.random.normal(size=1500)

coefs = coefs * np.sqrt(eigenvalues[:1500])
for ax, mode in zip(axes.flat, n_eigs):
    field = eigenvectors[:, :mode] @ coefs[:mode]

    pc = plot_mesh(mesh, fill_value=field, ax=ax, set_limits=True)
    # Add a caption with the value of n_eigs for each subplot
    ax.set_title(f"n_eigs = {mode}", fontsize=10)

cbar = fig.colorbar(
    pc,
    ax=ax,
    orientation="vertical",
    fraction=0.025,
    pad=0.04,
)
cbar.set_label("value", rotation=270, labelpad=20)
fig.suptitle("Matern 1/2 Prior", fontsize=16);